In [10]:
import tweepy
import pandas as pd

In [96]:
CONSUMER_KEY = CONSUMER_KEY
CONSUMER_SECRET = CONSUMER_SECRET
ACCESS_KEY = ACCESS_KEY
ACCESS_SECRET = ACCESS_SECRET

Authentication OK


In [109]:
CONSUMER_KEY = "LU6eEdt9a4PwQWIqnJtyVnIp9"
CONSUMER_SECRET = "jTCDms0tUgq91AQXXHKSzsuLorSSpSCPb4TTWOxfurrmk6q3gm"
ACCESS_KEY = "1372271424241602564-xIh4fpuVRXwrllCDSYdZyH8vDutziy"
ACCESS_SECRET = "OM6t5cBI5yOCVJd9uG3sFFmBmFpHQgvV2LWVP9mKkQDvN"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print('Authentication OK')
except:
    print('Error during authentication')

Authentication OK


In [3]:
# A test tweet object selected by tweet ID
tweet_by_id = api.get_status("1400703690290434049",tweet_mode = 'extended')

In [17]:
tweet_by_id._json['full_text']

{'created_at': 'Fri Jun 04 06:39:18 +0000 2021',
 'id': 1400703690290434049,
 'id_str': '1400703690290434049',
 'full_text': 'TRIGGER WARNING ⛔️ \n\nwhat you’re watching is the execution of Malcolm Johnson. he was murdered on March 31st by a KCPD officer. \n\nas you will see, THE FEMALE COP SHOT THE OTHER COP in the leg and then twice into Malcolm. KCPD claimed Malcom shot him. \n\nabolish KCPD. https://t.co/0v6eXqF6Q0',
 'truncated': False,
 'display_text_range': [0, 266],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 1400703621264777219,
    'id_str': '1400703621264777219',
    'indices': [267, 290],
    'media_url': 'http://pbs.twimg.com/ext_tw_video_thumb/1400703621264777219/pu/img/Em-A8krtliRxVUAY.jpg',
    'media_url_https': 'https://pbs.twimg.com/ext_tw_video_thumb/1400703621264777219/pu/img/Em-A8krtliRxVUAY.jpg',
    'url': 'https://t.co/0v6eXqF6Q0',
    'display_url': 'pic.twitter.com/0v6eXqF6Q0',
    'expanded_url': 'ht

## Twitter bot will search for replies to a certain tweet

In [136]:
def get_tweet_comments(input_tweet):

    """

    Args:
        input_tweet: A Twitter API response object

    Returns: A list of the text for all found replies.

    """
    comments = []
    name = input_tweet._json['user']['screen_name']
    tweet_id = input_tweet._json['id']
    for tweet in tweepy.Cursor(api.search,q=f"to:{name}", result_type = 'recent',tweet_mode = 'extended').items(100):
        if hasattr(tweet, 'in_reply_to_status_id_str'):
            if str(tweet.in_reply_to_status_id_str) == str(tweet_id):
                comments.append(tweet._json['full_text'])
                return tweet
    return comments

comments = get_tweet_comments(tweet_by_id)
print(comments)

Status(_api=<tweepy.api.API object at 0x000002408442CD00>, _json={'created_at': 'Thu Jun 10 00:39:25 +0000 2021', 'id': 1402787449214996482, 'id_str': '1402787449214996482', 'full_text': 'What city did this take place?', 'truncated': False, 'display_text_range': [0, 30], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://help.twitter.com/en/using-twitter/how-to-tweet#source-labels" rel="nofollow">twitter_apidemo33</a>', 'in_reply_to_status_id': 1402778894953062400, 'in_reply_to_status_id_str': '1402778894953062400', 'in_reply_to_user_id': 1372271424241602564, 'in_reply_to_user_id_str': '1372271424241602564', 'in_reply_to_screen_name': 'NathanFleck8', 'user': {'id': 1372271424241602564, 'id_str': '1372271424241602564', 'name': 'Nathan Fleck', 'screen_name': 'NathanFleck8', 'location': '', 'description': 'Love databases and numbers. Quant in the making.', 'url': None, 

In [8]:
# A test to print out up to 10 reply tweets
for x in comments[:10]:
    print(x)

@lmakjgb Now show the video of him killing the dude in 2016 or shooting at his gf earlier that day
@lmakjgb We need people to clean this trash off our streets
@lmakjgb @Chewyweeks This was spooky!!!
@lmakjgb Still don’t deserve to die
@lmakjgb @miguelmyers81 Then they panic and the Cover Up begins..
@lmakjgb What police said  the film doesn’t disrobe  The video shows the man attempting to flee police who tackle him off screen. Authorities said that Johnson shot an officer in the leg and another officer returned fire killing Johnson.
In both videos, it is unclear who fired the shots.
@lmakjgb What reason does she even have her weapon out for at that point?  If u need to pull a weapon and fumble that badly for it u should not be licensed to carry a weapon. So sick of this bullshit and I’m as white as Powder.
@lmakjgb smh
@lmakjgb This is frigging crazy eight cops and they cannot take down a suspect without murdering him. People are becoming so acclimated to cops killing people that the c

## Twitter bot will reply to the tweet

In [100]:
def twitter_reply(status_id, text):
    if text == "":
        reply_text = '''
        ------- test -------
        We noticed that you are reporting a police incident.
        '''
    else:
        reply_text = text

    api.update_status(status = reply_text, in_reply_to_status_id=status_id)
    return 'done!'

#twitter_reply(1395143654331744256)

In [12]:
df_data = []
for tweet in api.search(q='police', lang='en', rpp=5)[:5]:
    test_data = [tweet.user.screen_name, tweet.id, tweet.text]
    df_data.append(test_data)
df = pd.DataFrame(data=df_data, columns=['username', 'id', 'tweet'])
df

,username,id,tweet
0,ksmoola1,1402755452027523075,@patriotcanuck65 Not a gun in sight except fro...
1,NJ_Bronco_Fan,1402755451129942019,RT @MacFarlaneNews: FLASH: Georgia roofing co...
2,oneiriclillies,1402755447891939332,RT @hotties4dwt: what's happening in Venezuela...
3,Alejand14092042,1402755446444789771,@DailyYaeContent Dunno... if you kissed her......
4,victor2exl,1402755445257760773,RT @MaziNnamdiKanu: It’s pathetic that Nigeria...


## Twitter bot will reply to the user through DM

In [80]:
def direct_message(username):
    user = api.get_user(username)
    message = '''
    Just testing some features.
    This sends a direct message to the person
    '''
    api.send_direct_message(user.id, message)
    return 'direct message sent'

direct_message('@JoanVillar14')

'direct message sent'

Improvements:

1. Depending on the reply, gather more information such as:
- did it really happen?
- where did it happen

2. They pin the place of incident on a map
- If tweet's geolocation is on, get the coordinates

-------


* There are a few things to keep in mind in designing a model that replies to specific replies. This might be another project in the future.
* Since the incidents are already identified, it might be best to just get the location from the user.

In [154]:
class tweet_reachout:

    def __init__(self, tweet, open = True,  responded = True):
        self.initial_tweet_id = tweet.id
        self.open = open
        self.response = tweet
        self.conversation_log = [["Initial Tweet:", tweet._json['full_text']]]
        self.responded = responded

    # def respond(self):
    #     if self.responded:
    #         self.send_response()
    #         self.responded = False
    #     else:
    #         return "No response received"

    def send_response(self, text, id):
        #response_tweet_id = self.response.id
        twitter_reply(id,text)
        self.responded = True
        self.conversation_log.append(["Bot Response:", text])
        return "Response sent"

    def did_respond(self, tweet):
        self.response = tweet
        self.conversation_log.append([self.response.user.screen_name, self.response.full_text])
        self.responded = False

    def print_log(self):
        for i, x in enumerate(self.conversation_log):
            if i == 0 or x[0] == "Bot Response:":
                print(f"{x[0]}   {x[1]}")
            else:
                print(f"{x[0]} responded:   {x[1]}")

In [155]:
tweet_by_id = api.get_status("1402778894953062400",tweet_mode = 'extended')
print(tweet_by_id.id)

1402778894953062400


In [156]:
test_ticket = tweet_reachout(tweet_by_id)
test_ticket.print_log()

Initial Tweet:   test 1 2


In [157]:
test_ticket.send_response("What city did this take place?",tweet_by_id.id )

'Response sent'

In [158]:
test_ticket.print_log()

Initial Tweet:   test 1 2
Bot Response:   What city did this take place?


In [150]:
response = get_tweet_comments(tweet_by_id)

In [159]:
if response == []:
    v = 0
else:
    test_ticket.did_respond(response)
    test_ticket.send_response("Banana Town", response.id)

In [160]:
test_ticket.print_log()

Initial Tweet:   test 1 2
Bot Response:   What city did this take place?
NathanFleck8 responded:   Banana Town
Bot Response:   Banana Town


In [162]:
response = get_tweet_comments(api.get_status(str(response.id),tweet_mode = 'extended'))
print(response.id)

TweepError: [{'code': 144, 'message': 'No status found with that ID.'}]

In [163]:
if response == []:
    print("did not find response")
else:
    test_ticket.did_respond(response)
    test_ticket.send_response("Thank You", response.id)

In [ ]:
test_ticket.send_response("Hey man where is that at?","1402778894953062400")

In [ ]:
test_ticket.did_respond(api.get_status("1402307146503495681",tweet_mode = 'extended'))
test_ticket.send_response("Bruh chill")
test_ticket.did_respond(api.get_status("1402725931114909697",tweet_mode = 'extended'))

In [153]:
test_ticket.print_log()

Initial Tweet:   test 1 2
Bot Response:   What city did this take place?
NathanFleck8 responded:   What city did this take place?
Bot Response:   Banana Town
NathanFleck8 responded:   What city did this take place?
NathanFleck8 responded:   What city did this take place?
NathanFleck8 responded:   Banana Town
Bot Response:   Thank You


In [ ]:
1402787857274753026
